# 第7回講義 宿題

## 課題. Tensorflowを用いて, MNISTを畳み込みニューラルネットワーク(CNN)で学習せよ

### 注意

- homework関数を完成させて提出してください
    - 訓練データはtrain_X, train_y, テストデータはtest_Xで与えられます
    - train_Xとtrain_yをtrain_X, train_yとvalid_X, valid_yに分けるなどしてモデルを学習させてください
    - test_Xに対して予想ラベルpred_yを作り, homework関数の戻り値としてください\
- pred_yのtest_yに対する精度(F値)で評価します
- 全体の実行時間がiLect上で60分を超えないようにしてください
- homework関数の外には何も書かないでください (必要なものは全てhomework関数に入れてください)
- 解答提出時には Answer Cell の内容のみを提出してください

**`tf` の以下のモジュールはこの回では使用できないように制限されています. 注意してください.**
```python
tf.app
tf.compat
tf.contrib
tf.erros
tf.gfile
tf.graph_util
tf.image
tf.layers
tf.logging
tf.losses
tf.metrics
tf.python_io
tf.resource_loader
tf.saved_model
tf.sdca
tf.sets
tf.summary
tf.sysconfig
tf.test
```

次のセルのhomework関数を完成させて提出してください

# Answer Cell

In [9]:
def homework(train_X, train_y, test_X):
    import time
    import matplotlib.pyplot as plt
    #%matplotlib inline
    rng = np.random.RandomState(1234)
    random_state = 42
    #Convolution layers [batchsize,picture_length,picture_width,color_channels]
    class Conv:
        def __init__(self, filter_shape, function=lambda x: x, strides=[1,1,1,1], padding='VALID'):
            # Xavier Initialization
            fan_in = np.prod(filter_shape[:3])
            fan_out = np.prod(filter_shape[:2]) * filter_shape[3]
            self.W = tf.Variable(rng.uniform(
                            low=-np.sqrt(6/(fan_in + fan_out)),
                            high=np.sqrt(6/(fan_in + fan_out)),
                            size=filter_shape
                        ).astype('float32'), name='W')
            self.b = tf.Variable(np.zeros((filter_shape[3]), dtype='float32'), name='b')
            self.function = function
            self.strides = strides
            self.padding = padding

        def f_prop(self, x):
            u = tf.nn.conv2d(x, self.W, strides=self.strides, padding=self.padding)+self.b
            return self.function(u)
    class Conv_same:
        def __init__(self, filter_shape, function=lambda x: x, strides=[1,1,1,1], padding='SAME'):
            # Xavier Initialization
            fan_in = np.prod(filter_shape[:3])
            fan_out = np.prod(filter_shape[:2]) * filter_shape[3]
            self.W = tf.Variable(rng.uniform(
                            low=-np.sqrt(6/(fan_in + fan_out)),
                            high=np.sqrt(6/(fan_in + fan_out)),
                            size=filter_shape
                        ).astype('float32'), name='W')
            self.b = tf.Variable(np.zeros((filter_shape[3]), dtype='float32'), name='b')
            self.function = function
            self.strides = strides
            self.padding = padding

        def f_prop(self, x):
            u = tf.nn.conv2d(x, self.W, strides=self.strides, padding=self.padding)+self.b
            return self.function(u)
    #Pooling layers [batchsize,picture_length,picture_width,color_channels]
    class Pooling:
        def __init__(self, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME'):
            self.ksize = ksize
            self.strides = strides
            self.padding = padding

        def f_prop(self, x):
            return tf.nn.max_pool(x, ksize=self.ksize, strides=self.strides, padding=self.padding)
    #Flatten layer    
    class Flatten:
        def f_prop(self, x):
            return tf.reshape(x, (-1, np.prod(x.get_shape().as_list()[1:])))
    #Full-link layers
    class Dense:
        def __init__(self, in_dim, out_dim, function=lambda x: x):
            # Xavier Initialization
            self.W = tf.Variable(rng.uniform(
                            low=-np.sqrt(6/(in_dim + out_dim)),
                            high=np.sqrt(6/(in_dim + out_dim)),
                            size=(in_dim, out_dim)
                        ).astype('float32'), name='W')
            self.b = tf.Variable(np.zeros([out_dim]).astype('float32'))
            self.function = function

        def f_prop(self, x):
            return self.function(tf.matmul(x, self.W) + self.b)
    
    layers = [                            # (縦の次元数)x(横の次元数)x(チャネル数)
        Conv_same((5, 5, 1, 20), tf.nn.relu),  # 28x28x 1 -> 28x28x20
        Pooling((1, 2, 2, 1)),            # 28x28x20 -> 14x14x20
        Conv((5, 5, 20, 50), tf.nn.relu), # 14x14x20 ->  10x 10x50
        Pooling((1, 2, 2, 1)),            #  10x 10x50 ->  5x 5x50
        Flatten(),
        Dense(5*5*50, 10, tf.nn.softmax)
    ]

    x = tf.placeholder(tf.float32, [None, 28, 28, 1])
    t = tf.placeholder(tf.float32, [None, 10])

    def f_props(layers, x):
        for layer in layers:
            x = layer.f_prop(x)
        return x

    y = f_props(layers, x)

    cost = -tf.reduce_mean(tf.reduce_sum(t * tf.log(tf.clip_by_value(y, 1e-10, 1.0)), axis=1)) # tf.log(0)によるnanを防ぐ
    train = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

    valid = tf.argmax(y, 1)
    
    n_epochs = 200
    batch_size = 100
    n_batches = train_X.shape[0]//batch_size

    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    
    cost_plt =[]
    start_time=time.time()
    for epoch in range(n_epochs):
        train_X, train_y = shuffle(train_X, train_y, random_state=random_state)
        cost_arry = []
        for i in range(n_batches):
            start = i * batch_size
            end = start + batch_size
            _cost, _ = sess.run([cost,train], feed_dict={x: train_X[start:end], t: train_y[start:end]})
            cost_arry.append(_cost)
        cost_plt.append(np.sum(cost_arry))
        tttttt = time.time()-start_time
        print ("epoch",epoch,"time",tttttt,"cost",cost_plt[len(cost_plt)-1])

        pred_y = sess.run(valid, feed_dict={x: test_X})
        
    sess.close()
    plt.plot(np.log(cost_plt))
    
    return pred_y

- 以下のvalidate_homework関数を用いてエラーが起きないか動作確認をして下さい。
- 提出に際して、以下のscore_homework関数で60分で実行が終わることを確認して下さい。
- 評価は以下のscore_homework関数で行われますが、random_stateの値は変更されます。

# Checker Cell (for student)

In [10]:
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tensorflow.examples.tutorials.mnist import input_data

import numpy as np
import tensorflow as tf

del [
        tf.app,
        tf.compat,
        tf.contrib,
        tf.errors,
        tf.gfile,
        tf.graph_util,
        tf.image,
        tf.layers,
        tf.logging,
        tf.losses,
        tf.metrics,
        tf.python_io,
        tf.resource_loader,
        tf.saved_model,
        tf.sdca,
        tf.sets,
        tf.summary,
        tf.sysconfig,
        tf.test
]

def load_mnist():
    mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
    mnist_X = np.r_[mnist.train.images, mnist.test.images]
    mnist_y = np.r_[mnist.train.labels, mnist.test.labels]
    return train_test_split(mnist_X, mnist_y, test_size=0.2, random_state=42)

def validate_homework():
    train_X, test_X, train_y, test_y = load_mnist()
    train_X = train_X.reshape((train_X.shape[0], 28, 28, 1))
    test_X = test_X.reshape((test_X.shape[0], 28, 28, 1))

    # validate for small dataset
    train_X_mini = train_X[:5000]
    train_y_mini = train_y[:5000]
    test_X_mini = test_X[:1000]
    test_y_mini = test_y[:1000]

    pred_y = homework(train_X_mini, train_y_mini, test_X_mini)
    print(f1_score(np.argmax(test_y_mini, 1), pred_y, average='macro'))

def score_homework():
    train_X, test_X, train_y, test_y = load_mnist()
    train_X = train_X.reshape((train_X.shape[0], 28, 28, 1))
    test_X = test_X.reshape((test_X.shape[0], 28, 28, 1))
    
    pred_y = homework(train_X, train_y, test_X)
    print(f1_score(np.argmax(test_y, 1), pred_y, average='macro'))

AttributeError: app

In [11]:
#validate_homework()
#score_homework()

In [12]:
score_homework()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
epoch 0 time 3.3471519947052 cost 476.076
epoch 1 time 6.79206395149231 cost 142.758
epoch 2 time 10.220931768417358 cost 104.104
epoch 3 time 13.657320499420166 cost 83.3537
epoch 4 time 17.07565951347351 cost 70.0701
epoch 5 time 20.501519203186035 cost 60.6473
epoch 6 time 23.92888355255127 cost 54.0841
epoch 7 time 27.35875368118286 cost 49.056
epoch 8 time 30.791131734848022 cost 45.5713
epoch 9 time 34.23905110359192 cost 42.0872
epoch 10 time 37.707026720047 cost 39.3179
epoch 11 time 41.14742565155029 cost 37.0917
epoch 12 time 44.59534573554993 cost 35.2514
epoch 13 time 48.08588242530823 cost 33.2155
epoch 14 time 51.50267481803894 cost 32.0766
epoch 15 time 54.92503547668457 cost 30.7365
epoch 16 time 58.14843010902405 cost 29.3709
epoch 17 time 61.554248094558716 cost 28.5443
epoch 1

epoch 176 time 660.6979994773865 cost 2.33836
epoch 177 time 665.2198977470398 cost 2.21534
epoch 178 time 669.1790518760681 cost 2.20743
epoch 179 time 673.1483676433563 cost 2.21571
epoch 180 time 676.9381420612335 cost 2.18916
epoch 181 time 681.0135192871094 cost 2.05735
epoch 182 time 685.3940849304199 cost 2.13116
epoch 183 time 689.307867527008 cost 2.16634
epoch 184 time 693.2925717830658 cost 2.05956
epoch 185 time 697.5323371887207 cost 2.01453
epoch 186 time 701.3504157066345 cost 1.97886
epoch 187 time 705.4058990478516 cost 2.01828
epoch 188 time 709.6099400520325 cost 1.98744
epoch 189 time 712.910347700119 cost 1.91233
epoch 190 time 716.687947511673 cost 1.9133
epoch 191 time 720.3984217643738 cost 1.8875
epoch 192 time 724.3519954681396 cost 1.87477
epoch 193 time 727.9553666114807 cost 1.86611
epoch 194 time 731.4852619171143 cost 1.85252
epoch 195 time 735.1755728721619 cost 1.84128
epoch 196 time 738.7915601730347 cost 1.77593
epoch 197 time 742.2962183952332 cost 1

In [8]:
rng = np.random.RandomState(1234)
filter_shape=(5, 5, 1, 20)
fan_in = np.prod(filter_shape[:3])
fan_out = np.prod(filter_shape[:2]) * filter_shape[3]
print(fan_in)
print(fan_out)
print(rng.uniform(low=-np.sqrt(6/(fan_in + fan_out)),high=np.sqrt(6/(fan_in + fan_out)),size=filter_shape).shape)
b=np.zeros((filter_shape[3]), dtype='float32')

25
500
(5, 5, 1, 20)
